<a href="https://colab.research.google.com/github/Jhansipothabattula/Machine_Learning/blob/main/Day118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Simple FeedForward Neural Network

In [12]:
## Basic Neural Networks with TensorFlow

# Building a simple feedforward neural network

# 1. Import TensorFlow
import tensorflow as tf

# 2. Define the Architecture

# Example architecture for a simple neural network
input_size = 784  # (e.g., pixels in an image)
hidden_size = 128 # Number of neurons in the hidden layer
output_size = 10  # (e.g., digits 0-9)

# 3. Specify the Input and Output Layers
X = tf.keras.Input(shape=(input_size,), dtype=tf.float32, name='input_features')
y = tf.keras.Input(shape=(output_size,), dtype=tf.float32, name='target_labels')

# 4. Configure the Hidden Layers

# Hidden layer
hidden_layer = tf.keras.layers.Dense(units=hidden_size, activation=tf.nn.relu)(X)

# 5. Specify the Output Layer

# Output layer
output_layer = tf.keras.layers.Dense(units=output_size)(hidden_layer)

# Define the Keras Model
model = tf.keras.Model(inputs=X, outputs=output_layer, name='feedforward_nn')

# 6. Define the Loss Function
class SoftmaxCrossEntropyWithLogitsLayer(tf.keras.layers.Layer):
    def call(self, labels, logits):
        return tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

# Wrap tf.reduce_mean in a Keras Layer to accept KerasTensors.
class ReduceMeanLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.reduce_mean(inputs)

# 7. Select the Optimization Algorithm

# Adam optimizer for gradient descent (TF2.x version)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# 8. Initialize Variables and Training Loop (TF2.x Eager Execution)

# Define some dummy data for the example (as in previous code)
num_epochs = 10
input_data = tf.random.normal((100, input_size))
target_labels = tf.one_hot(tf.random.uniform((100,), maxval=output_size, dtype=tf.int32), depth=output_size)

# Build the model by passing dummy input to initialize weights
_ = model(tf.zeros((1, input_size)))

# Get trainable variables from the model after it's built
trainable_vars = model.trainable_variables

# For evaluation, use a Keras metric
accuracy_metric = tf.keras.metrics.CategoricalAccuracy()
test_data = tf.random.normal((10, input_size))
test_labels = tf.one_hot(tf.random.uniform((10,), maxval=output_size, dtype=tf.int32), depth=output_size)

print("Starting training")
for epoch in range(num_epochs):
    with tf.GradientTape() as tape:
        # Forward pass
        logits = model(input_data) # Use the Keras model directly

        # Calculate loss using the custom Keras layers
        cross_entropy = SoftmaxCrossEntropyWithLogitsLayer()(labels=target_labels, logits=logits)
        current_loss = ReduceMeanLayer()(cross_entropy)

    # Compute gradients
    gradients = tape.gradient(current_loss, trainable_vars)

    # Apply gradients
    optimizer.apply_gradients(zip(gradients, trainable_vars))

    # Print training loss (using .numpy() to get scalar value from a TF tensor)
    print(f"Epoch {epoch+1}, Loss: {current_loss.numpy()}")

print("Training finished.")

# 9. Evaluate the Model (Optional)

# Perform evaluation in eager mode
test_logits = model(test_data)

# Update accuracy metric state
accuracy_metric.update_state(y_true=test_labels, y_pred=tf.nn.softmax(test_logits))

# Get the final accuracy result
test_accuracy = accuracy_metric.result().numpy()
print("Test Accuracy:", test_accuracy)

Starting training
Epoch 1, Loss: 3.109400987625122
Epoch 2, Loss: 2.392617702484131
Epoch 3, Loss: 1.771527886390686
Epoch 4, Loss: 1.2569451332092285
Epoch 5, Loss: 0.857225239276886
Epoch 6, Loss: 0.5674097537994385
Epoch 7, Loss: 0.3691628575325012
Epoch 8, Loss: 0.240563303232193
Epoch 9, Loss: 0.15901751816272736
Epoch 10, Loss: 0.1076737493276596
Training finished.
Test Accuracy: 0.1
